# Spin Rate Analysis: Impact on Golf Ball Trajectory

This notebook analyzes how spin rate affects carry distance and trajectory shape using our Magnus effect model.

In [ ]:
import sys
sys.path.append('../code')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from magnus_simulation import GolfBallSimulator, LaunchConditions, mph_to_ms, meters_to_yards

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12

## 1. Effect of Spin Rate on Carry Distance

We'll simulate shots with varying spin rates to see how backspin affects carry distance.

In [ ]:
# Simulation parameters
sim = GolfBallSimulator()

# Fixed conditions (typical PGA Tour driver)
ball_speed = mph_to_ms(165)  # 165 mph
launch_angle = 11.0  # degrees

# Vary spin rate
spin_rates = np.arange(1500, 4500, 100)  # rpm
results = []

for spin in spin_rates:
    conditions = LaunchConditions(
        ball_speed=ball_speed,
        launch_angle=launch_angle,
        launch_direction=0,
        spin_rate=spin,
        spin_axis=0
    )
    
    sim.simulate(conditions)
    
    results.append({
        'spin_rate': spin,
        'carry_yards': meters_to_yards(sim.get_carry_distance()),
        'max_height_yards': meters_to_yards(sim.get_max_height()),
        'flight_time': sim.get_flight_time()
    })

df = pd.DataFrame(results)
df.head()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Carry distance vs spin
axes[0].plot(df['spin_rate'], df['carry_yards'], 'b-', linewidth=2)
axes[0].axvline(x=2500, color='red', linestyle='--', alpha=0.7, label='Optimal (~2500 rpm)')
axes[0].set_xlabel('Spin Rate (rpm)')
axes[0].set_ylabel('Carry Distance (yards)')
axes[0].set_title('Carry Distance vs Spin Rate')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Max height vs spin
axes[1].plot(df['spin_rate'], df['max_height_yards'], 'g-', linewidth=2)
axes[1].set_xlabel('Spin Rate (rpm)')
axes[1].set_ylabel('Maximum Height (yards)')
axes[1].set_title('Ball Flight Height vs Spin Rate')
axes[1].grid(True, alpha=0.3)

# Flight time vs spin
axes[2].plot(df['spin_rate'], df['flight_time'], 'm-', linewidth=2)
axes[2].set_xlabel('Spin Rate (rpm)')
axes[2].set_ylabel('Flight Time (seconds)')
axes[2].set_title('Flight Time vs Spin Rate')
axes[2].grid(True, alpha=0.3)

plt.suptitle(f'Effect of Backspin on Driver Performance\n(Ball Speed: 165 mph, Launch Angle: 11°)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/spin_rate_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. Spin Axis Effect: Draw vs Fade

Analyze how spin axis tilt affects lateral deviation.

In [ ]:
# Fixed conditions
conditions = LaunchConditions(
    ball_speed=mph_to_ms(165),
    launch_angle=11.0,
    launch_direction=0,
    spin_rate=2500,
    spin_axis=0
)

# Vary spin axis
spin_axes = np.arange(-20, 21, 2)  # degrees
axis_results = []

for axis in spin_axes:
    conditions.spin_axis = axis
    sim.simulate(conditions)
    
    axis_results.append({
        'spin_axis': axis,
        'lateral_yards': meters_to_yards(sim.get_lateral_deviation()),
        'carry_yards': meters_to_yards(sim.get_carry_distance())
    })

df_axis = pd.DataFrame(axis_results)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Lateral deviation
axes[0].plot(df_axis['spin_axis'], df_axis['lateral_yards'], 'b-', linewidth=2, marker='o')
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0].fill_between(df_axis['spin_axis'], 0, df_axis['lateral_yards'], 
                     where=df_axis['lateral_yards'] > 0, alpha=0.3, color='blue', label='Fade (Right)')
axes[0].fill_between(df_axis['spin_axis'], 0, df_axis['lateral_yards'], 
                     where=df_axis['lateral_yards'] < 0, alpha=0.3, color='red', label='Draw (Left)')
axes[0].set_xlabel('Spin Axis (degrees)')
axes[0].set_ylabel('Lateral Deviation (yards)')
axes[0].set_title('Spin Axis vs Lateral Deviation')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Carry distance (shows slight loss with sidespin)
axes[1].plot(df_axis['spin_axis'], df_axis['carry_yards'], 'g-', linewidth=2, marker='o')
axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Spin Axis (degrees)')
axes[1].set_ylabel('Carry Distance (yards)')
axes[1].set_title('Carry Distance vs Spin Axis')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Magnus Effect: Spin Axis Influence on Ball Flight', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/spin_axis_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Model Validation with PGA Tour Data

Compare our model predictions with published PGA Tour averages.

In [ ]:
# PGA Tour average driver statistics (2023-2024)
pga_data = {
    'club': ['Driver', '3-Wood', '5-Iron', '7-Iron', 'PW'],
    'ball_speed_mph': [171, 158, 132, 120, 102],
    'launch_angle': [10.4, 9.3, 14.1, 16.3, 24.2],
    'spin_rate': [2545, 3655, 4693, 7124, 9304],
    'carry_actual': [275, 243, 195, 172, 136]  # yards
}

pga_df = pd.DataFrame(pga_data)

# Simulate each club
predicted_carry = []

for _, row in pga_df.iterrows():
    conditions = LaunchConditions(
        ball_speed=mph_to_ms(row['ball_speed_mph']),
        launch_angle=row['launch_angle'],
        launch_direction=0,
        spin_rate=row['spin_rate'],
        spin_axis=0
    )
    
    sim.simulate(conditions)
    predicted_carry.append(meters_to_yards(sim.get_carry_distance()))

pga_df['carry_predicted'] = predicted_carry
pga_df['error_yards'] = pga_df['carry_predicted'] - pga_df['carry_actual']
pga_df['error_percent'] = (pga_df['error_yards'] / pga_df['carry_actual']) * 100

print("Model Validation Against PGA Tour Averages")
print("=" * 60)
print(pga_df[['club', 'carry_actual', 'carry_predicted', 'error_yards', 'error_percent']].to_string(index=False))
print(f"\nMean Absolute Error: {abs(pga_df['error_yards']).mean():.1f} yards")
print(f"Mean Absolute % Error: {abs(pga_df['error_percent']).mean():.1f}%")

In [ ]:
# Validation plot
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(pga_df['carry_actual'], pga_df['carry_predicted'], s=100, c='blue', alpha=0.7)

# Perfect prediction line
line_range = [100, 300]
ax.plot(line_range, line_range, 'k--', label='Perfect Prediction')

# Add labels for each club
for _, row in pga_df.iterrows():
    ax.annotate(row['club'], (row['carry_actual'], row['carry_predicted']),
                textcoords='offset points', xytext=(5, 5), fontsize=10)

ax.set_xlabel('Actual Carry Distance (yards)')
ax.set_ylabel('Predicted Carry Distance (yards)')
ax.set_title('Model Validation: Predicted vs Actual PGA Tour Distances')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

plt.tight_layout()
plt.savefig('../figures/model_validation.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Key Findings Summary

Based on our analysis:

1. **Optimal Backspin:** For driver shots at ~165 mph ball speed, optimal spin is approximately 2000-2500 rpm
2. **Spin Axis Sensitivity:** Each degree of spin axis tilt causes approximately X yards of lateral deviation
3. **Model Accuracy:** Our Magnus effect model predicts carry distances within Y% of PGA Tour averages